## Difference in LLM and Agent

- **LLM**: Just predicts text based on input.

- **Agent**: Uses LLM + tools to reason, decide, and act (e.g., call APIs, fetch data).

### 1. Load the LLM Model
#### Option 1: Using Hugging Face FLAN-T5
- This model is not as capable and may sometimes confuse tool usage, resulting in incorrect or no tool invocation.

#### Option 2: Using Gemini 2.0 Flash from Google (via LangChain)
 - This uses Google's Gemini model via the `langchain_google_genai` wrapper. It is better at tool routing and faster, suitable for tool-using agents.

In [95]:
# from transformers import pipeline
# from langchain_huggingface.llms import HuggingFacePipeline

# pipe = pipeline(
#     "text2text-generation",
#     model="google/flan-t5-base",
#     tokenizer=LLM_MODEL_ID,
#     max_new_tokens=256,
#     do_sample=False,
#     device=-1  # CPU; set to 0 for GPU if available
# )
# llm = HuggingFacePipeline(pipeline=pipe)

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    max_tokens=256,             # Max length of the response
)


### 2.a Custom Tool using `@tool` decorator from `langchain_core.tools`
- This approach auto-wraps the function and exposes it as a LangChain-compatible Tool.

- The tool metadata (like name and description) can be added by setting attributes after definition.

- Ideal for direct integration with LangChain’s newer core API.



In [96]:
from langchain_core.tools import tool
import requests

@tool
def get_dummyjson_products(limit: int = 10) -> str:
    """
    Fetch products from DummyJSON API.
    Parameters:
        limit: number of products to return
    Returns:
        A string summary of products.
    """
    url = f"https://dummyjson.com/products?limit={limit}"
    response = requests.get(url)
    if response.ok:
        products = response.json().get("products", [])
        summaries = [f"{p['title']} (${p['price']})" for p in products]
        return "Products:\n" + "\n".join(summaries)
    else:
        return "Error fetching products."

# Manually assign name and description to make it compatible with ReAct agents
get_dummyjson_products.description = "Fetch products from DummyJSON API. Input: number of products."
get_dummyjson_products.name = "get_dummyjson_products"

# Try invoking the tool manually
get_dummyjson_products.invoke({"limit": 6})

'Products:\nEssence Mascara Lash Princess ($9.99)\nEyeshadow Palette with Mirror ($19.99)\nPowder Canister ($14.99)\nRed Lipstick ($12.99)\nRed Nail Polish ($8.99)\nCalvin Klein CK One ($49.99)'

### 2.b Tool Creation Using `Tool` Class
- This is the classic LangChain way of defining tools explicitly.

- You pass the function, name, and description into the Tool constructor.

- Recommended when you want full control over tool metadata or are using legacy agents.

In [121]:
from langchain.agents import Tool

def get_stock_price(symbol: str) -> float:
    return {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "AMZN": 150.0,
        "RIL": 87.6
    }.get(symbol, 0.0)

stock_price_tool = Tool(
    name="get_stock_price",
    func=get_stock_price,
    description="Get the current price of a stock"
)

# Try invoking the tool manually
stock_price_tool.invoke({"symbol": "MSFT"})

200.3

In [122]:
from langchain.agents import load_tools

custom_tools = [get_dummyjson_products, stock_price_tool]
builtin_wikipedia_tool = load_tools(["wikipedia"], llm = llm)

tools = custom_tools + builtin_wikipedia_tool

In [123]:
for t in tools:
    print(f"{t.name}: {type(t)}")

get_dummyjson_products: <class 'langchain_core.tools.structured.StructuredTool'>
get_stock_price: <class 'langchain_core.tools.simple.Tool'>
wikipedia: <class 'langchain_community.tools.wikipedia.tool.WikipediaQueryRun'>


### 3. Create the Agent

#### `initialize_agent()` vs `create_react_agent()`
##### initialize_agent():
- Higher level builder
- Its like a "smart shortcut" that sets up an agent for you automatically based on the AgentType.
- Handles tool loading, prompt setup, memory, and agent creation automatically.
- Requires minimal configuration.
- Uses create_react_agent() behind the scenes (when using ReAct-based types).

##### create_react_agent():
- Lower-level, 
- Gives you fine-grained control over the exact ReAct-style agent that’s created.
- You must handle: Prompt creation (or let it use default), Tool list, Agent construction
- No agent executor is returned — you need to wrap the result using `AgentExecutor`.

#### When to use what?

- I just want tools + LLM to work quickly with ReAct logic -> initialize_agent()

- I want to create a custom prompt, intercept intermediate steps, or fine-tune agent internals -> create_react_agent() + AgentExecutor


### 3.a Using `create_react_agent()`
The create_react_agent function calls `.partial(...)` on your prompt, automatically injecting:

`tools = tools_renderer(...)` <br />
`tool_names = ", ".join([t.name for t in tools])`

So you should NOT declare tools or tool_names inside input_variables — just make sure {tools} and {tool_names} are present in the template string.

In [120]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

template = """Answer the following question as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original question

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input","agent_scratchpad"]
)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 3.b Using `initialize_agent()`
What is ZERO_SHOT_REACT_DESCRIPTION?
- A ReAct-style agent: It can **reason and act** step-by-step.
- Zero-shot: It doesn’t need few-shot examples — it relies only on the tool descriptions and the query.
- It reads tool descriptions and decides which tool(s) to use for answering the user's question.

In [101]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)


### Define queries

In [109]:
query1 = 'Show me 6 products from DummyJSON.'
query2 = 'What is the current GDP of India?'
query3 = 'What is the current share price of MSFT?'

### 4.a Invoke 1st agent

In [116]:
agent_executor.invoke({'input': query1})




> Entering new AgentExecutor chain...
Thought: I need to use the get_dummyjson_products tool to get the products.
Action: get_dummyjson_products
Action Input: 6Products:
Essence Mascara Lash Princess ($9.99)
Eyeshadow Palette with Mirror ($19.99)
Powder Canister ($14.99)
Red Lipstick ($12.99)
Red Nail Polish ($8.99)
Calvin Klein CK One ($49.99)I now know the final answer
Final Answer: Essence Mascara Lash Princess ($9.99), Eyeshadow Palette with Mirror ($19.99), Powder Canister ($14.99), Red Lipstick ($12.99), Red Nail Polish ($8.99), Calvin Klein CK One ($49.99)


> Finished chain.


{'input': 'Show me 6 products from DummyJSON.',
 'output': 'Essence Mascara Lash Princess ($9.99), Eyeshadow Palette with Mirror ($19.99), Powder Canister ($14.99), Red Lipstick ($12.99), Red Nail Polish ($8.99), Calvin Klein CK One ($49.99)'}

In [117]:
agent_executor.invoke({'input': query2})




> Entering new AgentExecutor chain...
Thought: I need to find the GDP of India. I can use Wikipedia to search for this information.
Action: wikipedia
Action Input: GDP of IndiaPage: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and

{'input': 'What is the current GDP of India?',
 'output': "I cannot provide the exact current GDP of India with the available tools. However, India is the world's fourth-largest economy by nominal GDP."}

In [118]:
agent_executor.invoke({'input': query3})



> Entering new AgentExecutor chain...
Thought: I need to find the current share price of Microsoft, which has the stock symbol MSFT.
Action: get_stock_price
Action Input: MSFT200.3I now know the final answer
Final Answer: 200.3


> Finished chain.


{'input': 'What is the current share price of MSFT?', 'output': '200.3'}

### 4.b Invoke 2nd agent

In [113]:
response = agent.run(query1)
print(f"Query 1: {query1}\nResponse: {response}")



> Entering new AgentExecutor chain...
I need to fetch 6 products from the DummyJSON API.
Action: get_dummyjson_products
Action Input: 6
Observation: Products:
Essence Mascara Lash Princess ($9.99)
Eyeshadow Palette with Mirror ($19.99)
Powder Canister ($14.99)
Red Lipstick ($12.99)
Red Nail Polish ($8.99)
Calvin Klein CK One ($49.99)
Thought:I now know the final answer
Final Answer: Essence Mascara Lash Princess ($9.99), Eyeshadow Palette with Mirror ($19.99), Powder Canister ($14.99), Red Lipstick ($12.99), Red Nail Polish ($8.99), Calvin Klein CK One ($49.99)

> Finished chain.
Query 1: Show me 6 products from DummyJSON.
Response: Essence Mascara Lash Princess ($9.99), Eyeshadow Palette with Mirror ($19.99), Powder Canister ($14.99), Red Lipstick ($12.99), Red Nail Polish ($8.99), Calvin Klein CK One ($49.99)


In [119]:

response = agent.run(query2)
print(f"Query 2: {query2}\n{response}")




> Entering new AgentExecutor chain...
I need to find the current GDP of India. I can use Wikipedia to search for this information.
Action: wikipedia
Action Input: "GDP of India"
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisati

In [115]:
response = agent.run(query3)
print(f"Query 2: {query2}\n{response}")



> Entering new AgentExecutor chain...
I need to find the current share price of MSFT, which is the stock ticker for Microsoft. I can use the get_stock_price tool for this.
Action: get_stock_price
Action Input: MSFT
Observation: 200.3
Thought:I now know the final answer
Final Answer: 200.3

> Finished chain.
Query 2: What is the current GDP of India?
200.3
